# 評価例：プッシュ通知バルク実験

評価は**タスク指向**で反復的であり、LLM統合の状況を確認し改善するための最良の方法です。

以下の評価では、**モデルとプロンプトの多くのバリエーションをテストする**タスクに焦点を当てます。

私たちのユースケースは：
1. プッシュ通知要約機能から可能な限り最高のパフォーマンスを得たい

## 評価の構造

評価には「Eval」と「Run」の2つの部分があります。「Eval」はテスト基準の設定とあなたの「Run」のデータ構造を保持します。Evalは複数のrunを`has_many`で持ち、それらはあなたのテスト基準によって評価されます。

In [1]:
import pydantic
import openai
from openai.types.chat import ChatCompletion
import os

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "your-api-key")

## ユースケース

以下の統合機能をテストしています。これはプッシュ通知要約機能で、複数のプッシュ通知を受け取り、それらを1つのメッセージにまとめます。

In [ ]:
class PushNotifications(pydantic.BaseModel):
    notifications: str

print(PushNotifications.model_json_schema())

In [ ]:
DEVELOPER_PROMPT = """
You are a helpful assistant that summarizes push notifications.
You are given a list of push notifications and you need to collapse them into a single one.
Output only the final summary, nothing else.
"""

def summarize_push_notification(push_notifications: str) -> ChatCompletion:
    result = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "developer", "content": DEVELOPER_PROMPT},
            {"role": "user", "content": push_notifications},
        ],
    )
    return result

example_push_notifications_list = PushNotifications(notifications="""
- Alert: Unauthorized login attempt detected.
- New comment on your blog post: "Great insights!"
- Tonight's dinner recipe: Pasta Primavera.
""")
result = summarize_push_notification(example_push_notifications_list.notifications)
print(result.choices[0].message.content)

# Evalの設定

Evalは複数の*Run*間で共有される設定を保持し、2つのコンポーネントで構成されています：

1. データソース設定 `data_source_config` - 将来の*Run*が準拠するスキーマ（列）。
    - `data_source_config`はJSON Schemaを使用して、Evalで利用可能な変数を定義します。
2. テスト基準 `testing_criteria` - データソースの各*行*に対して、統合が機能しているかどうかを判断する方法。

このユースケースでは、プッシュ通知サマリーの完了が良好かどうかをテストしたいので、これを念頭に置いてevalを設定します。

In [4]:
# We want our input data to be available in our variables, so we set the item_schema to
# PushNotifications.model_json_schema()
data_source_config = {
    "type": "custom",
    "item_schema": PushNotifications.model_json_schema(),
    # We're going to be uploading completions from the API, so we tell the Eval to expect this
    "include_sample_schema": True,
}

この`data_source_config`は、eval全体で利用可能な変数を定義します。

このアイテムスキーマ：
```json
{
  "properties": {
    "notifications": {
      "title": "Notifications",
      "type": "string"
    }
  },
  "required": ["notifications"],
  "title": "PushNotifications",
  "type": "object"
}
```
これは、evalで変数`{{item.notifications}}`が利用可能になることを意味します。

`"include_sample_schema": True`
これは、evalで変数`{{sample.output_text}}`が利用可能になることを意味します。

**それでは、これらの変数を使用してテスト条件を設定します。**

In [5]:
GRADER_DEVELOPER_PROMPT = """
Categorize the following push notification summary into the following categories:
1. concise-and-snappy
2. drops-important-information
3. verbose
4. unclear
5. obscures-meaning
6. other 

You'll be given the original list of push notifications and the summary like this:

<push_notifications>
...notificationlist...
</push_notifications>
<summary>
...summary...
</summary>

You should only pick one of the categories above, pick the one which most closely matches and why.
"""
GRADER_TEMPLATE_PROMPT = """
<push_notifications>{{item.notifications}}</push_notifications>
<summary>{{sample.output_text}}</summary>
"""
push_notification_grader = {
    "name": "Push Notification Summary Grader",
    "type": "label_model",
    "model": "o3-mini",
    "input": [
        {
            "role": "developer",
            "content": GRADER_DEVELOPER_PROMPT,
        },
        {
            "role": "user",
            "content": GRADER_TEMPLATE_PROMPT,
        },
    ],
    "passing_labels": ["concise-and-snappy"],
    "labels": [
        "concise-and-snappy",
        "drops-important-information",
        "verbose",
        "unclear",
        "obscures-meaning",
        "other",
    ],
}

`push_notification_grader`は、入力`{{item.notifications}}`と生成された要約`{{sample.output_text}}`を確認し、「correct」または「incorrect」としてラベル付けするモデルグレーダー（llm-as-a-judge）です。
その後、「passing_labels」を通じて、合格となる回答の条件を指示します。

注意：内部的には構造化出力を使用しているため、ラベルは常に有効です。

**それでは評価を作成し、データの追加を開始しましょう！**

In [6]:
eval_create_result = openai.evals.create(
    name="Push Notification Bulk Experimentation Eval",
    metadata={
        "description": "This eval tests many prompts and models to find the best performing combination.",
    },
    data_source_config=data_source_config,
    testing_criteria=[push_notification_grader],
)
eval_id = eval_create_result.id

# 実行の作成

テスト基準（testing_criteria）を使用した評価のセットアップが完了したので、多数の実行を追加し始めることができます！
まずはプッシュ通知データから始めましょう。

In [7]:
push_notification_data = [
        """
- New message from Sarah: "Can you call me later?"
- Your package has been delivered!
- Flash sale: 20% off electronics for the next 2 hours!
""",
        """
- Weather alert: Thunderstorm expected in your area.
- Reminder: Doctor's appointment at 3 PM.
- John liked your photo on Instagram.
""",
        """
- Breaking News: Local elections results are in.
- Your daily workout summary is ready.
- Check out your weekly screen time report.
""",
        """
- Your ride is arriving in 2 minutes.
- Grocery order has been shipped.
- Don't miss the season finale of your favorite show tonight!
""",
        """
- Event reminder: Concert starts at 7 PM.
- Your favorite team just scored!
- Flashback: Memories from 3 years ago.
""",
        """
- Low battery alert: Charge your device.
- Your friend Mike is nearby.
- New episode of "The Tech Hour" podcast is live!
""",
        """
- System update available.
- Monthly billing statement is ready.
- Your next meeting starts in 15 minutes.
""",
        """
- Alert: Unauthorized login attempt detected.
- New comment on your blog post: "Great insights!"
- Tonight's dinner recipe: Pasta Primavera.
""",
        """
- Special offer: Free coffee with any breakfast order.
- Your flight has been delayed by 30 minutes.
- New movie release: "Adventures Beyond" now streaming.
""",
        """
- Traffic alert: Accident reported on Main Street.
- Package out for delivery: Expected by 5 PM.
- New friend suggestion: Connect with Emma.
"""]

今度は、テストするためのプロンプトをいくつか設定していきます。

基本的なプロンプトを、いくつかのバリエーションでテストしたいと思います：
1. 1つ目のバリエーションでは、基本的なプロンプトのみを使用します
2. 2つ目では、要約がどのような形になってほしいかの良い例をいくつか含めます
3. 最後のバリエーションでは、良い例と悪い例の両方を含めます。

また、使用するモデルのリストも含めます。

In [8]:
PROMPT_PREFIX = """
You are a helpful assistant that takes in an array of push notifications and returns a collapsed summary of them.
The push notification will be provided as follows:
<push_notifications>
...notificationlist...
</push_notifications>

You should return just the summary and nothing else.
"""

PROMPT_VARIATION_BASIC = f"""
{PROMPT_PREFIX}

You should return a summary that is concise and snappy.
"""

PROMPT_VARIATION_WITH_EXAMPLES = f"""
{PROMPT_VARIATION_BASIC}

Here is an example of a good summary:
<push_notifications>
- Traffic alert: Accident reported on Main Street.- Package out for delivery: Expected by 5 PM.- New friend suggestion: Connect with Emma.
</push_notifications>
<summary>
Traffic alert, package expected by 5pm, suggestion for new friend (Emily).
</summary>
"""

PROMPT_VARIATION_WITH_NEGATIVE_EXAMPLES = f"""
{PROMPT_VARIATION_WITH_EXAMPLES}

Here is an example of a bad summary:
<push_notifications>
- Traffic alert: Accident reported on Main Street.- Package out for delivery: Expected by 5 PM.- New friend suggestion: Connect with Emma.
</push_notifications>
<summary>
Traffic alert reported on main street. You have a package that will arrive by 5pm, Emily is a new friend suggested for you.
</summary>
"""

prompts = [
    ("basic", PROMPT_VARIATION_BASIC),
    ("with_examples", PROMPT_VARIATION_WITH_EXAMPLES),
    ("with_negative_examples", PROMPT_VARIATION_WITH_NEGATIVE_EXAMPLES),
]

models = ["gpt-4o", "gpt-4o-mini", "o3-mini"]

**これで、すべてのプロンプトとすべてのモデルをループして、複数の設定を一度にテストできます！**

プッシュ通知リストにテンプレート変数を使用して、'completion'実行データソースを使用します。

OpenAIが補完呼び出しを処理し、"sample.output_text"を入力してくれます。

In [ ]:
for prompt_name, prompt in prompts:
    for model in models:
        run_data_source = {
            "type": "completions",
            "input_messages": {
                "type": "template",
                "template": [
                    {
                        "role": "developer",
                        "content": prompt,
                    },
                    {
                        "role": "user",
                        "content": "<push_notifications>{{item.notifications}}</push_notifications>",
                    },
                ],
            },
            "model": model,
            "source": {
                "type": "file_content",
                "content": [
                    {
                        "item": PushNotifications(notifications=notification).model_dump()
                    }
                    for notification in push_notification_data
                ],
            },
        }

        run_create_result = openai.evals.runs.create(
            eval_id=eval_id,
            name=f"bulk_{prompt_name}_{model}",
            data_source=run_data_source,
        )
        print(f"Report URL {model}, {prompt_name}:", run_create_result.report_url)


## おめでとうございます。データセット全体で9つの異なるプロンプトとモデルのバリエーションをテストしました！